# Setup

Make sure you're in the 'owen' directory

In [3]:
# %cd ..
%ls

Adapter.ipynb  __pycache__/                 Terminal commands.txt
Optimus_dir/   run_latent_generation.ipynb


In [1]:
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM, Trainer, LlamaTokenizer
import datasets
import matplotlib.pyplot as plt
import torch.nn as nn
from tqdm import tqdm
import torch
import numpy as np

# import accelerate

In [1]:
# del Optimus
from Optimus_dir.code.Optimus import Optimus

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [10]:
# initialize Optimus VAE
vae32 = Optimus('Optimus_dir/code/optimus_latent32_beta05', 32)

Model name 'Optimus_dir/code/optimus_latent32_beta05/checkpoint-encoder-508523' was not found in model name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc). We assumed 'Optimus_dir/code/optimus_latent32_beta05/checkpoint-encoder-508523' was a path or url but couldn't find any file associated to this path or url.


OSError: file Optimus_dir/code/optimus_latent32_beta05/checkpoint-encoder-508523 not found

In [4]:
vae = Optimus('Optimus_dir/code/optimus_latent768_beta05', 768)

We have added 3 tokens to GPT2


In [ ]:
vae_beta1 = Optimus('Optimus_dir/code/optimus_latent768_beta1', 768)

In [11]:
# Test optimus
text_original = "After playing college basketball for the LSU Tigers, O'Neal was drafted by the Orlando Magic with the first overall pick in the 1992 NBA draft. He quickly became one of the best centers in the league, winning Rookie of the Year in 1992–93 and leading his team to the 1995 NBA Finals."
latent, code = vae.latent_code_from_text(text_original)
print(len(text_original))
# text = vae.text_from_latent_code(latent)
for i in range(10):
    print(vae.text_from_latent_code(latent))

283
He was selected 21st overall by the San Diego Padres in the 2004 amateur draft and received his first major league footy starting at shortstop, where he batted.306, hitting.385 with five home runs, 38 RBI and 67 runs scored.
After coaching for the Minnesota Lynx of the National Basketball Association, Hanley competed for his college club in the 2014 NCAA tournament, appearing in four seasons with the Orlando Magic, Chicago Bulls and Philadelphia 76ers.
In 1996, he started as a guard and enrolled in Georgia Southern University, where he quickly transferred to defensive backs at the start of the 1997 season, where he played his freshman season of 2008, helping Georgia Southern upset Georgia Tech 19–7.
Columns and other major professional leagues was filled by Filipino draft choice Rodolfo Francisco, who was drafted by Kenosha High School in the 4th round (32nd overall) of the 1985 USA National League, where he played his first professional baseball as a puncher in 1998.
In 1985, he m

Load model

- 9.3s with 5cpus 0gpus

In [7]:
# model_name_or_path = "ehartford/Wizard-Vicuna-30B-Uncensored"
# model_name_or_path = "mistralai/Mistral-7B-Instruct-v0.1"
# model_name_or_path = "EleutherAI/pythia-410m"
model_name_or_path = "openlm-research/open_llama_3b_v2"
# model_name_or_path = "EleutherAI/pythia-1.4b"
# !huggingface-cli login
# model_name_or_path = "meta-llama/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto")
# model.to(torch.float16)
# use_fast_tokenizer = "LlamaForCausalLM" not in model.config.architectures
use_fast_tokenizer = True #OpenLlama v2 docs says use LlamaTokenizer directly
# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=use_fast_tokenizer, padding_side="left", legacy=False)
tokenizer = LlamaTokenizer.from_pretrained(model_name_or_path, use_fast=use_fast_tokenizer)
# tokenizer.pad_token_id = 0

Test model

- 21.2s for 5cpus 0 gpus(max new tokens = 32)

In [8]:
prompt = 'Q: What is the largest country?\nA:'
# tokenizer, model = accelerator.prepare(tokenizer, model)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

generation_output = model.generate(
    input_ids=input_ids, max_new_tokens=32
)
print(tokenizer.decode(generation_output[0]))

<s>Q: What is the largest country?
A: The largest country is Russia.
Q: What is the smallest country?
A: The smallest country is Vatican City.
Q: What is


Load and check dataset (mini pile)
- 1.2s with 5cpu 0 gpu

In [12]:
dataset = datasets.load_dataset("NeelNanda/pile-10k", split="train").remove_columns("meta")
print(dataset)
print(dataset[0]['text'][:200])

Dataset({
    features: ['text'],
    num_rows: 10000
})
It is done, and submitted. You can play “Survival of the Tastiest” on Android, and on the web. Playing on the web works, but you have to simulate multi-touch for table moving and that can be a bit con


Initialize adapter

In [13]:
# Create the model class for adapter, should just be a linear from dim of VAE latent to 
# class Adapter(nn.Linear):
#     def __init__(self, d_in, d_out):
#         super().__init__()
        

#     def forward(self, tokens: Int[Tensor, "batch position"]) -> Float[Tensor, "batch position d_vocab"]:
#         return 
d_vae_latent = 32 # Latent size, could also be 768
d_lm_embedding = model.config.hidden_size
adapter = nn.Linear(d_vae_latent, d_lm_embedding)

In [ ]:
class AdapterTrainer(Trainer):
    def __init__(self, vae, lm):
        # Trainer for an adapter from vae to lm
        super().__init__()
        self.vae = vae
        self.lm = lm

    def vae_encode(self, inputs):
        latent_z, coded_length = self.vae.latent_code_from_text(inputs,)
        return latent_z

    def lm_hidden(self, text = None, vec = None):
        '''
        Returns the hidden full state of the LLM given either text or a vector
        Hidden state is a tuple of FloatTensors, one for each hidden layer (including embedding)
        Only confirmed to work for Llama architecture (incl OpenLlama)
        '''
        if text is not None:

            return self.lm(input_ids = text, output_hidden_states=True, return_dict=True).hidden_states
        elif vec is not None:
            
            return self.lm(inputs_embeds=vec, output_hidden_states=True, return_dict=True).hidden_states
        else:
            raise ValueError("Must provide either text or vec")

    def compute_loss(self, model, inputs, return_outputs=False):

        #1. Get VAE-encoded input
        vae_encoded_inputs = self.vae_encode(inputs)
        
        #2. Pass VAE latent vector into adapter (get "output" from "model")
        outputs = model(**vae_encoded_inputs)
        
        #3. Get hidden state of LM using adapter output as input
        hidden_state_from_adapter = self.lm_hidden(vec = outputs) 

        #4. Pass the same text into LLM and get hidden state
        hidden_state_from_text = self.lm_hidden(text = inputs)
        
        #5. Compute loss with cosine similarity between hidden states 
        loss = nn.functional.cosine_similarity(hidden_state_from_adapter, hidden_state_from_text) 
        # I think need to make sure hidden states are both 1d vectors
        
        return (loss, outputs) if return_outputs else loss

In [ ]:
# Initialize trainer and run loop